In [1]:
import sys
sys.path.append('../../')
from util import get_loss_data, get_original_datasets, get_models, get_train_test
from model_config import generate_model_configs, get_parameters, ModelConfig, ParameterConfig
from loss_config import get_loss_config

loss_datas = get_loss_data(path="../../data_generation/data")
original_datasets = get_original_datasets(path="../../data_generation/datasets")
models = get_models()
print(models)

loss_config = get_loss_config(default_lambda=True)
model_configs: ModelConfig = generate_model_configs(models, loss_config)
params: ParameterConfig = get_parameters(path="../../params.yml")
print(params)
print(model_configs)

/home/tollef/miniconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


{'gte-base': 'thenlper/gte-base', 'gte-small': 'thenlper/gte-small', 'e5-small': 'intfloat/e5-small-v2', 'minilm-6': 'sentence-transformers/all-MiniLM-L6-v2'}
ParameterConfig(RANDOM_SEED=42, SAMPLING_SEED=42, VERBOSE=False, N_ITERS=5, EVAL_EVERY=2, BATCH_SIZE=64, DEVICE='cuda', K=16, LEARNING_RATE=3e-05)
{'Triplet(gte-base)': ModelConfig(model=None, model_name='thenlper/gte-base', loss=<class 'sentence_transformers.losses.TripletLoss.TripletLoss'>, loss_params={}), 'Contrastive(gte-base)': ModelConfig(model=None, model_name='thenlper/gte-base', loss=<class 'sentence_transformers.losses.ContrastiveLoss.ContrastiveLoss'>, loss_params={}), 'OnlineContrastive(gte-base)': ModelConfig(model=None, model_name='thenlper/gte-base', loss=<class 'sentence_transformers.losses.OnlineContrastiveLoss.OnlineContrastiveLoss'>, loss_params={}), 'MultipleNegatives(gte-base)': ModelConfig(model=None, model_name='thenlper/gte-base', loss=<class 'sentence_transformers.losses.MultipleNegativesRankingLoss.Mult

In [2]:
# from model_evaluator import evaluate_model
# from tqdm import tqdm
# from util import get_train_test
# from sentence_transformers import SentenceTransformer

# all_scores = {}
# k_values = [4, 8, 16, 32, 64]
# for dataset in original_datasets.keys():
#     all_scores[dataset] = {}
#     for model_id, model_config in model_configs.items():
#         print(f"Evaluating {model_id}")
#         model_name = model_config.model_name
#         model = SentenceTransformer(model_name).to(params.DEVICE)
#         train, test = get_train_test(original_datasets, dataset)
#         ref_train_emb = model.encode(train['text'].tolist())
#         ref_test_emb = model.encode(test['text'].tolist())
#         all_scores[dataset][model_name] = {}
#         for k in tqdm(k_values):
#             scores = evaluate_model(
#                 model=model,
#                 name=f"{dataset}_{model_id}",
#                 train_data=train,
#                 test_data=test,
#                 reference_train_emb=ref_train_emb,
#                 reference_test_emb=ref_test_emb,
#                 k=k,
#                 verbose=False
#             )
#             all_scores[dataset][model_name][k] = scores

In [3]:
# import pandas as pd
# import numpy as np

# data = []
# for dataset, dataset_scores in all_scores.items():
#     for model_name, model_scores in dataset_scores.items():
#         for k, scores in model_scores.items():
#             polarity = scores['polarity']
#             p_mean = np.round(np.mean(polarity), 3)
#             p_std = np.round(np.std(polarity), 3)
#             semantic = scores['semantic']
#             s_mean = np.round(np.mean(semantic), 3)
#             s_std = np.round(np.std(semantic), 3)

#             row = {
#                 'dataset': dataset,
#                 'model': model_name,
#                 'k': k,
#                 'p_mean': p_mean,
#                 'p_std': p_std,
#                 's_mean': s_mean,
#                 's_std': s_std
#             }
#             data.append(row)

# df = pd.DataFrame(data)
# model_translations = {v: k for k, v in models.items()}
# df.model = df.model.apply(lambda x: model_translations[x])
# df.to_csv('baseline_k.csv', index=False)

In [4]:
# load the df
import pandas as pd

df = pd.read_csv('baseline_k.csv')

In [5]:
from IPython.display import display


k_16 = df[df.k == 16].reset_index().drop(columns=['index', 'k'])
k_16 = k_16.set_index(['model']).sort_index().round(3)

from collections import defaultdict
latex_rows = defaultdict(list)  # For Tables 5 and 6 in Section 4.
k_16 = k_16.sort_values(by=['model', 'dataset'])
latex_rows["polarity"] = k_16.apply(lambda x: f"{x['p_mean']}_{{{x['p_std']}}}", axis=1).values
latex_rows["semantic"] = k_16.apply(lambda x: f"{x['s_mean']}_{{{x['s_std']}}}", axis=1).values

for metric, latex_row in latex_rows.items():
    print(metric)
    latex_row = [f"${x}$" for x in latex_row]
    print(f"Reference & - & {' & '.join(latex_row)} \\\\")
    
display(k_16)

for model in sorted(models.keys()):
    model_df = k_16.loc[model]
    model_df = model_df.round(3)
    # for m in ["polarity", "semantic"]:
    #     display(model_df[[f"{m}_mean", f"{m}_std"]].to_latex())
    display(model_df)

polarity
Reference & - & $71.436_{21.241}$ & $81.465_{23.696}$ & $67.402_{20.737}$ & $80.376_{22.561}$ & $66.826_{20.573}$ & $77.774_{22.204}$ & $63.751_{20.178}$ & $63.016_{21.877}$ \\
semantic
Reference & - & $83.404_{1.463}$ & $85.516_{1.735}$ & $81.366_{1.641}$ & $83.7_{1.436}$ & $82.468_{1.629}$ & $84.808_{1.41}$ & $42.307_{5.615}$ & $46.594_{7.426}$ \\


,dataset,p_mean,p_std,s_mean,s_std
model,,,,,
e5-small,sarcastic-headlines,71.436,21.241,83.404,1.463
e5-small,sst2,81.465,23.696,85.516,1.735
gte-base,sarcastic-headlines,67.402,20.737,81.366,1.641
gte-base,sst2,80.376,22.561,83.700,1.436
gte-small,sarcastic-headlines,66.826,20.573,82.468,1.629
gte-small,sst2,77.774,22.204,84.808,1.410
minilm-6,sarcastic-headlines,63.751,20.178,42.307,5.615
minilm-6,sst2,63.016,21.877,46.594,7.426


,dataset,p_mean,p_std,s_mean,s_std
model,,,,,
e5-small,sarcastic-headlines,71.436,21.241,83.404,1.463
e5-small,sst2,81.465,23.696,85.516,1.735


,dataset,p_mean,p_std,s_mean,s_std
model,,,,,
gte-base,sarcastic-headlines,67.402,20.737,81.366,1.641
gte-base,sst2,80.376,22.561,83.700,1.436


,dataset,p_mean,p_std,s_mean,s_std
model,,,,,
gte-small,sarcastic-headlines,66.826,20.573,82.468,1.629
gte-small,sst2,77.774,22.204,84.808,1.410


,dataset,p_mean,p_std,s_mean,s_std
model,,,,,
minilm-6,sarcastic-headlines,63.751,20.178,42.307,5.615
minilm-6,sst2,63.016,21.877,46.594,7.426


In [6]:
k_16[k_16.dataset == 'sarcastic-headlines']

,dataset,p_mean,p_std,s_mean,s_std
model,,,,,
e5-small,sarcastic-headlines,71.436,21.241,83.404,1.463
gte-base,sarcastic-headlines,67.402,20.737,81.366,1.641
gte-small,sarcastic-headlines,66.826,20.573,82.468,1.629
minilm-6,sarcastic-headlines,63.751,20.178,42.307,5.615
